In [2]:
import pandas as pd
import json 
import gzip
import os
import re as regex
import progressbar
import pickle

In [11]:
product_path = '../../../../src/data/product'
train_test_all_filtered_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables/large/after_manual_checking')
files_representation_train = [file for file in os.listdir(os.path.join(train_test_all_filtered_path,'test')) if file.endswith('.json.gz')]
turl_input_path = os.path.join(product_path, 'TURL/input')

In [12]:
files_representation_train

['Product_sixtysixtysounds.com_September2020.json.gz',
 'Product_ahorroimprimiendo.com_September2020.json.gz',
 'Product_watch4usale.com_September2020.json.gz',
 'Product_novmak.com_September2020.json.gz',
 'Product_camtecphoto.com_September2020.json.gz',
 'Product_reveresecurity.com_September2020.json.gz',
 'Product_comboink.com_September2020.json.gz',
 'Product_roobol.com_September2020.json.gz',
 'Product_roysbikes.com_September2020.json.gz',
 'Product_weaverscarpet.com_September2020.json.gz',
 'Product_louisianarunning.com_September2020.json.gz',
 'Product_torichristgen.com_September2020.json.gz',
 'Product_heathflooring.com_September2020.json.gz',
 'Product_yudinsappliances.com_September2020.json.gz',
 'Product_velopro.com_September2020.json.gz',
 'Product_billmans.com_September2020.json.gz',
 'Product_frenshmo.com.au_September2020.json.gz',
 'Product_kinetikcycles.com_September2020.json.gz',
 'Product_beandcare.com_September2020.json.gz',
 'Product_welovetec.com_September2020.json

In [39]:
#clean mapping files and save them again
# then use the cleaned tables to append into a series and the get a dataframe from that
count = 0
train_representation=[] #representation of whole training set
with progressbar.ProgressBar(max_value=len(files_representation_train)) as bar:
    for zip_file in files_representation_train:
        print('/{}'.format(zip_file))
        df = pd.read_json(os.path.join(train_test_all_filtered_path,'test') + '/{}'.format(zip_file), compression='gzip', lines=True)
        if ('description' in df.columns)== False: # check if description column given
            df['description'] = ''
        #if ('name' in df.columns)== False: # check if description column given
         #   df['name'] = x = df.page_url.apply(lambda x: x.split("/")[4])
        df['header']=''#empty string for headers
        table_representation = [] #empty list for table
        table_representation.append(zip_file)#append table id
        table_representation.append('')#append page title -> not relevant
        table_representation.append('')#append wikipedia page id -> not given
        table_representation.append('')#append information about entity -> product, not relevant since same for all tables
        table_representation.append('')#append table caption -> not given
        table_representation.append(df['header'].to_list()) #append headers -> not sure if we should do that
        all_rows_representation = []#representation of all rows
        for i in range(len(df)):
            row_representation=[] # cell representation of single row
            row_representation.append([[0,i],[df['row_id'][i],str(df['name'][i])]])
            row_representation.append([[1,i],[df['row_id'][i],str(df['description'][i])]])
            all_rows_representation.append(row_representation) #append single cell representation to representation of all rows
        table_representation.append(all_rows_representation)#append it to representation of whole table
        train_representation.append(table_representation)
        count += 1
        bar.update(count)

  0% (0 of 1687) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

/Product_sixtysixtysounds.com_September2020.json.gz
/Product_ahorroimprimiendo.com_September2020.json.gz
/Product_watch4usale.com_September2020.json.gz
/Product_novmak.com_September2020.json.gz
/Product_camtecphoto.com_September2020.json.gz


  0% (5 of 1687) |                       | Elapsed Time: 0:00:00 ETA:   0:00:46

/Product_reveresecurity.com_September2020.json.gz
/Product_comboink.com_September2020.json.gz
/Product_roobol.com_September2020.json.gz


  0% (9 of 1687) |                       | Elapsed Time: 0:00:00 ETA:   0:01:48

...

/Product_allmountaincyclery.com_September2020.json.gz
/Product_trunkclothiers.com_September2020.json.gz
/Product_homedecoraz.com_September2020.json.gz
/Product_heinnie.com_September2020.json.gz


 99% (1679 of 1687) |################### | Elapsed Time: 0:02:30 ETA:   0:00:00

/Product_yepmebazar.com_September2020.json.gz
/Product_consommables.com_September2020.json.gz
/Product_ventadecomputadoras.com.mx_September2020.json.gz
/Product_richstv.com_September2020.json.gz
/Product_aveneusa.com_September2020.json.gz
/Product_curlycooart.com_September2020.json.gz
/Product_petimce.com_September2020.json.gz


100% (1687 of 1687) |####################| Elapsed Time: 0:02:30 Time:  0:02:30


/Product_aktivwholesale.com_September2020.json.gz
/Product_specialisteduski.com_September2020.json.gz
/Product_thecraftist.com_September2020.json.gz
/Product_area399.com_September2020.json.gz


In [40]:
with open(os.path.join(turl_input_path, 'test_representation.txt'), "wb") as fp:  #Pickling
    pickle.dump(train_representation, fp)

In [7]:
with open(os.path.join(turl_input_path, 'test_representation.txt'), "rb") as fp:   # Unpickling
    train_representation = pickle.load(fp)